# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv('output_data/output.csv')
city_df.head()


,Unnamed: 0,City,Lat,Lng,temp,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
0,0,boa vista,2.82,-60.67,80.6,75,BR,1592438482,83,80.6,2.24
1,1,saldanha,-33.01,17.94,50.0,11,ZA,1592438644,71,50.0,3.36
2,2,jamestown,42.10,-79.24,78.8,1,US,1592438529,41,78.8,11.41
3,3,albany,42.60,-73.97,82.0,0,US,1592438644,47,82.0,1.88
4,4,hilo,19.73,-155.09,84.2,1,US,1592438640,54,84.2,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [9]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Create New DataFrame to narrow down the cities to fit weather conditions
fit_weather_df = city_df.loc[city_df['Max Temp'] < 70]
fit_weather_df = fit_weather_df.loc[fit_weather_df['Wind Speed'] < 13]
fit_weather_df

,Unnamed: 0,City,Lat,Lng,temp,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
1,1,saldanha,-33.01,17.94,50.00,11,ZA,1592438644,71,50.00,3.36
6,6,barrow,71.29,-156.79,42.80,1,US,1592438645,70,42.80,12.75
9,9,ushuaia,-54.80,-68.30,35.60,57,AR,1592438529,80,35.60,6.93
10,10,mount gambier,-37.83,140.77,48.20,66,AU,1592438646,81,48.20,12.75
13,13,san martin,-33.08,-68.47,50.00,38,AR,1592438646,57,50.00,3.36
...,...,...,...,...,...,...,...,...,...,...,...
534,534,puerto madryn,-42.77,-65.04,53.15,97,AR,1592438726,50,53.15,9.86
535,535,vitim,59.45,112.56,63.50,68,RU,1592438726,61,63.50,1.41
539,539,ayan,56.45,138.17,42.67,98,RU,1592438726,96,42.67,4.83
541,541,kalmar,56.66,16.36,52.00,4,SE,1592438727,100,52.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = fit_weather_df

params = {
    "type" : "lodging",
    "radius" : 5000,
    "key" : g_key
}

#use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():

    params['location'] = f"{row['Lat']}, {row['Lng']}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {row['Lat']}, {row['Lng']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 1: -33.01, 17.94.
Closest hotel is Blue Bay Lodge.
------------
Retrieving Results for Index 6: 71.29, -156.79.
Closest hotel is King Eider Inn.
------------
Retrieving Results for Index 9: -54.8, -68.3.
Closest hotel is Albatros Hotel.
------------
Retrieving Results for Index 10: -37.83, 140.77.
Closest hotel is Southgate Motel.
------------
Retrieving Results for Index 13: -33.08, -68.47.
Closest hotel is Finca Vivencias.
------------
Retrieving Results for Index 15: -46.19, 168.86.
Closest hotel is Ellie's Villa.
------------
Retrieving Results for Index 17: 37.73, -25.67.
Closest hotel is Vila Nova Hotel.
------------
Retrieving Results for Index 19: 6.27, 21.22.
Missing field/result... skipping.
------------
Retrieving Results for Index 20: -46.6, 168.33.
Closest hotel is Bluff Homestead - Guesthouse & Campervan Park.
------------
Retrieving Results for Index 21: 64.89, -23.71.
Closest hotel is Hotel Olafsvik.
------------
Retrieving Results for Index

Closest hotel is Nuevo Hotel Plaza.
------------
Retrieving Results for Index 180: 67.47, 78.7.
Closest hotel is TazHostel.
------------
Retrieving Results for Index 181: 45.25, 77.98.
Closest hotel is Gostinitsa "Diyar".
------------
Retrieving Results for Index 182: -45.42, 167.72.
Closest hotel is Kingsgate Hotel Te Anau.
------------
Retrieving Results for Index 184: 63.46, 48.9.
Missing field/result... skipping.
------------
Retrieving Results for Index 185: 29.11, 21.29.
Missing field/result... skipping.
------------
Retrieving Results for Index 198: 38.73, -27.07.
Closest hotel is Hotel Branco I.
------------
Retrieving Results for Index 200: 64.75, 177.48.
Closest hotel is Гостиница "Анадырь".
------------
Retrieving Results for Index 201: 52.01, 106.86.
Closest hotel is Selenga.
------------
Retrieving Results for Index 202: -46.28, 169.85.
Closest hotel is Kaitangata Motor Camp.
------------
Retrieving Results for Index 205: 71.04, 27.85.
Closest hotel is Mehamn Arctic Hotel 

Closest hotel is Angora Hotel.
------------
Retrieving Results for Index 367: 43.61, -116.39.
Closest hotel is Courtyard by Marriott Boise West/Meridian.
------------
Retrieving Results for Index 368: 43.27, -7.54.
Closest hotel is Pazo de Verdes.
------------
Retrieving Results for Index 372: 32.3, -9.24.
Closest hotel is Hôtel Abda.
------------
Retrieving Results for Index 376: 59.12, 31.68.
Closest hotel is Ptk.
------------
Retrieving Results for Index 377: -19.98, 57.61.
Closest hotel is Pereybere Hotel & Spa.
------------
Retrieving Results for Index 379: 61.56, 23.6.
Closest hotel is Hotel Lamminpää.
------------
Retrieving Results for Index 380: 60.92, -46.05.
Closest hotel is Hotel Narsaq.
------------
Retrieving Results for Index 381: 70.47, -68.59.
Closest hotel is Qamaq Hotel.
------------
Retrieving Results for Index 385: -33.29, 23.49.
Closest hotel is Jumper's Place.
------------
Retrieving Results for Index 387: 36.62, 101.77.
Closest hotel is Super Leisure Hotel.
----

Closest hotel is GreenTree Hotel Rongcheng Bus Station.
------------
Retrieving Results for Index 531: 60.4, -1.35.
Closest hotel is Busta House Hotel.
------------
Retrieving Results for Index 532: 52.54, 11.98.
Closest hotel is Schlosshotel Tangermünde.
------------
Retrieving Results for Index 534: -42.77, -65.04.
Closest hotel is Hotel Tolosa.
------------
Retrieving Results for Index 535: 59.45, 112.56.
Closest hotel is Gostinitsa V Apteke.
------------
Retrieving Results for Index 539: 56.45, 138.17.
Closest hotel is Gostinitsa.
------------
Retrieving Results for Index 541: 56.66, 16.36.
Closest hotel is First Hotel Witt.
------------
Retrieving Results for Index 546: 41.96, 42.96.
Closest hotel is კურორტი ზეკარი/Resort Zekari.
------------


In [38]:
hotel_df

,Unnamed: 0,City,Lat,Lng,temp,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed,Hotel Name,name
1,1,saldanha,-33.01,17.94,50.00,11,ZA,1592438644,71,50.00,3.36,Blue Bay Lodge,Blue Bay Lodge
6,6,barrow,71.29,-156.79,42.80,1,US,1592438645,70,42.80,12.75,King Eider Inn,King Eider Inn
9,9,ushuaia,-54.80,-68.30,35.60,57,AR,1592438529,80,35.60,6.93,Albatros Hotel,Albatros Hotel
10,10,mount gambier,-37.83,140.77,48.20,66,AU,1592438646,81,48.20,12.75,Southgate Motel,Southgate Motel
13,13,san martin,-33.08,-68.47,50.00,38,AR,1592438646,57,50.00,3.36,Finca Vivencias,Finca Vivencias
...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,534,puerto madryn,-42.77,-65.04,53.15,97,AR,1592438726,50,53.15,9.86,Hotel Tolosa,Hotel Tolosa
535,535,vitim,59.45,112.56,63.50,68,RU,1592438726,61,63.50,1.41,Gostinitsa V Apteke,Gostinitsa V Apteke
539,539,ayan,56.45,138.17,42.67,98,RU,1592438726,96,42.67,4.83,Gostinitsa,Gostinitsa
541,541,kalmar,56.66,16.36,52.00,4,SE,1592438727,100,52.00,2.24,First Hotel Witt,First Hotel Witt


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.heatmap_layer(locations,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))